# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

### Загрузка корпуса

In [5]:
import pickle
import pymorphy2

In [2]:
with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)
    
len(qa_corpus)

1384

In [3]:
questions = list()
answers = list()

for item in qa_corpus:
    questions.append(item[0])
    answers.append(item[1])

### Препроцессинг

In [185]:
morph = pymorphy2.MorphAnalyzer()

def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

# Индексация

In [89]:
import numpy as np
from judicial_splitter import splitter as sp
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from math import log
from gensim import matutils

## Обратный индекс

In [11]:
# собираем списки лемм для вопросов и ответов
questions_lemmas = list()
answers_lemmas = list()

for pair in tqdm(qa_corpus):
    questions_lemmas.append(preprocessing(pair[0]))
    answers_lemmas.append(preprocessing(pair[1]))

100%|██████████████████████████████████████| 1384/1384 [05:23<00:00,  4.28it/s]


In [13]:
# делаем из списков лемм строки для работы CountVectorizer
questions_joint = list()
answers_joint = list()

for doc in tqdm(questions_lemmas):
    questions_joint.append(' '.join(doc))
    
for doc in tqdm(answers_lemmas):
    answers_joint.append(' '.join(doc))

100%|███████████████████████████████████| 1384/1384 [00:00<00:00, 44609.28it/s]


In [14]:
# обучаем CountVectorizer
CV = CountVectorizer()
q_vec = CV.fit_transform(questions_joint)
q_df = pd.DataFrame(q_vec.toarray(), columns=CV.get_feature_names())

a_vec = CV.fit_transform(answers_joint)
a_df = pd.DataFrame(a_vec.toarray(), columns=CV.get_feature_names())

a_df.head()

,00,0001,01,02,03,04,05,06,07,08,...,январялюдмилаи,ярлык,ярослав,ярославский,ясна,ясно,яхта,ячейка,ящик,ёрш
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
def save_inverted_index_base(corpus, names) -> dict:
    """
    Create inverted index by input doc collection
    :param corpus: list: input doc collection
    :param names: list: list of names for input doc collection
    :return: inverted index
    """
    joint_lemmas = list()

    for doc in tqdm(corpus):
        joint_lemmas.append(' '.join(preprocessing(doc)))

    corpus_vec = CV.fit_transform(joint_lemmas)
    corpus_df = pd.DataFrame(corpus_vec.toarray(), columns=CV.get_feature_names())
    
    index = defaultdict()
    
    for col in tqdm(corpus_df):
        index[col] = [names[i] for i in list(corpus_df[col][corpus_df[col] > 0].index)]
    
    return index

In [26]:
# сохраняем обратный индекс для вопросов и ответов
inv_q_base = save_inverted_index_base(questions, list(range(0, len(questions))))
inv_a_base = save_inverted_index_base(answers, list(range(0, len(answers))))

100%|██████████████████████████████████████| 1384/1384 [01:54<00:00, 12.12it/s]
7955it [00:21, 362.71it/s]                                                     
100%|██████████████████████████████████████| 1384/1384 [04:01<00:00,  5.73it/s]
8928it [00:24, 362.28it/s]                                                     


In [30]:
def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    idf = log((N - n + 0.5) / (n + 0.5))
    score = (idf * (k1 + 1) * qf) / (qf + k1 * (1 - b + b * dl / avgdl))
        
    return score

def compute_sim(query, doc, inv_index, k1, b, avgdl, N) -> float:
    """
    Compute parameters for BM25 score and pass them to the calculation function
    :param query: str: word for which to claculate BM25
    :param doc: str: doc for which to claculate BM25
    :param inv_index: default_dict: inverted index for the collection, that includes doc
    :return: score
    """
    qf = doc.count(query)
    dl = len(doc)
    
    if query in inv_index:
        n = len(inv_index[query])
    else:
        n = 0
     
    return score_BM25(qf, dl, avgdl, k1, b, N, n)

## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [56]:
from gensim.models import Word2Vec, KeyedVectors

c:\users\masha\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [57]:
model_w2v = Word2Vec.load(r'araneum_none_fasttextskipgram_300_5_2018/araneum_none_fasttextskipgram_300_5_2018.model')

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [73]:
def get_w2v_vectors(doc):
    """
    Get doc w2v vector
    :param doc: str: doc for which to compute vector
    :return: list: cpmputed vector
    """
    all_vect = list()
    for word in doc:
        try:
            all_vect.append(model_w2v.wv[word])
        except:
            continue
            
    if len(all_vect) != 0:
        d_vect = np.mean(np.array(all_vect), axis=0)
    else:
        d_vect = [0]*300
        
    return d_vect 

def save_w2v_base(texts, idx):
    """
    Save vectors for all passed documents
    :param texts: list: documents of collection 
    :param idx: list: names of documants from collection 
    :return: list: list of vectors for input text
    """
    base = list()
    for i, doc in tqdm(enumerate(texts)):
        parts = sp(doc, 3)
        for p in parts:
            lemmas = preprocessing(p)
            base.append({'id': idx[i], 'text': p, 'vec': get_w2v_vectors(lemmas)})
    return base

In [77]:
# сохраняем w2v базу для вопросов и ответов
w2v_base_quest = save_w2v_base(questions, list(range(0, len(questions))))
w2v_base_answ = save_w2v_base(answers, list(range(0, len(answers))))

1384it [01:55, 12.02it/s]
1384it [03:51,  5.98it/s]


## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [112]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

In [12]:
# собираем пути документов коллекции
names = list()
for root, dirs, files in os.walk('./article'):
    for name in tqdm(files):
        names.append(os.path.join(root, name))

100%|███████████████████████████████| 224756/224756 [00:03<00:00, 58841.02it/s]


In [35]:
def preprocessing_d2v(text, lemma=True):
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(text)]
    if lemma is True:
        words = [morph.parse(x)[0].normal_form for x in words if x]    
    return words

In [40]:
def train_doc2vec(names):
    '''
    Train custome d2v model
    :param names: pathes of the train data docs
    :return: d2v model
    '''
    tagged_data = list()
    for path in tqdm(names):
        with open(path, 'r', encoding='utf-8') as f:
            text = f.read()
            splitted = sp(text, 3)
            for part in splitted:                
                tagged_data.append(TaggedDocument(words=preprocessing_d2v(part, lemma=False), tags=[path]))
    
    model_d2v = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100, workers=4, dm=1, seed=42)
    model_d2v.build_vocab(tagged_data)
    print(len(model_d2v.wv.vocab))
    
    model_d2v.train(tagged_data, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs, report_delay=60)
    return model_d2v

In [ ]:
model_d2v = train_doc2vec(names)

In [ ]:
fname = get_tmpfile("doc2vec_model_judic")
model_d2v.save(fname)

In [81]:
# модель не обучилась за 1,5 суток, загружем готовую
model_d2v = Doc2Vec.load('Doc2Vec_100s_1000e.bin')

In [146]:
model_d2v.random.seed(42)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [82]:
def get_d2v_vectors(words):
    '''
    Compute d2v vector for doc
    :param words: list: lemmas of the doc for which to compute d2v vector
    :return: list: d2v vector
    '''
    vec = model_d2v.infer_vector(words)
    return vec 

def save_d2v_base(docs, idx):
    """
    Save d2v vectors for all passed documents
    :param texts: list: documents of collection 
    :param idx: list: names of documants from collection 
    :return: list: list of d2v vectors for input text
    """
    base = list()
    for i, doc in tqdm(enumerate(docs)):
        parts = sp(doc, 3)
        for p in parts:
            lemmas = preprocessing(p)
            base.append({'id': idx[i], 'text': p, 'vect': get_d2v_vectors(lemmas)})
    return base

In [85]:
# сохраняем d2v базу для вопросов и ответов
d2v_base_quest = save_d2v_base(questions, list(range(0, len(questions))))
d2v_base_answ = save_d2v_base(answers, list(range(0, len(answers))))

1384it [07:48,  2.95it/s]
1384it [15:05,  1.53it/s]


# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [127]:
def similarity(v1, v2):
    '''
    Compute cosine similarity for 2 vectors
    :param v1, v2: list: vectors
    :return: float: vectors' similarity
    '''
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    sim = np.dot(v1_norm, v2_norm)
    if sim is not None:
        return sim
    else:
        return 0

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [215]:
def search_inv(query, questions, answers, inv_index) -> list:
    """
    Search documents relative to query using inverted index algorithm.
    :param query: str: input text
    :param questions: list: all questions from corpus
    :param answers: list: all answers from corpus
    :param inv_index: list: questions inverted index
    :return: list: 5 relative answers
    """
    k1 = 2.0
    b = 0.75
    avgdl = np.mean(list(map(len, questions)))
    N = len(questions)
    
    query_list = preprocessing(query)
    scores = list()
    
    for i, doc in enumerate(questions):
        score = 0
        for word in query_list:
            score += compute_sim(word, doc, inv_index, k1, b, avgdl, N)
        scores.append([i, score])
        
    ranked = sorted(scores, key = lambda x: x[1], reverse=True)
    result = [{'id': doc[0], 'text': answers[doc[0]]} for doc in ranked[:5]]

    return result

def search_w2v(query, w2v_base_quest, answers) -> list:
    """
    Search documents relative to query using inverted w2v algorithm.
    :param query: str: input text
    :param w2v_base_quest: list: all questions' vectors from corpus
    :param answers: list: all answers from corpus
    :return: list: 5 relative answers
    """
    
    similarities = list()

    for part in sp(query, 3):
        lemmas = preprocessing(query)
        vec = get_w2v_vectors(lemmas)
    
        for quest in w2v_base_quest:
            s = similarity(vec, quest['vec'])
            similarities.append({'id': quest['id'], 'sim': s})

    ranked = sorted(similarities, key=lambda x: x['sim'], reverse=True)
    result = [{'id': doc['id'], 'text': answers[doc['id']]} for doc in ranked[:5]]
    
    return result
    
def search_d2v(query, d2v_base_quest, answers) -> list:
    """
    Search documents relative to query using inverted d2v algorithm.
    :param query: str: input text
    :param d2v_base_quest: list: all questions' vectors from corpus
    :param answers: list: all answers from corpus
    :return: list: 5 relative answers
    """
    similarities = list()

    for part in sp(query, 3):
        lemmas = preprocessing(query)
        vec = get_d2v_vectors(lemmas)
    
        for quest in d2v_base_quest:
            s = similarity(vec, quest['vect'])
            similarities.append({'id': quest['id'], 'sim': s})

    ranked = sorted(similarities, key=lambda x: x['sim'], reverse=True)
    result = [{'id': doc['id'], 'text': answers[doc['id']]} for doc in ranked[:5]]   
    
    return result

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [96]:
def search(query, search_method):
    if search_method == 'inverted_index':
        search_result = search_inv(query, questions, answers, inv_q_base)
    elif search_method == 'word2vec':
        search_result = search_w2v(query, w2v_base_quest, answers)
    elif search_method == 'doc2vec':
        search_result = search_d2v(query, d2v_base_quest, answers)
    else:
        raise TypeError('unsupported search method')
    return search_result

## Оценка качества методов поиска

Выдача считается "успешной", если среди первых пяти ответов, предложенных методом поиска есть настоящий ответ.

In [110]:
def eval_search_method(search_method):  
    '''
    Evaluate search method
    :param search_method: search method to be evaluated
    :return: float: share of the rigth answers
    '''
    success = 0
    for i, q in tqdm(enumerate(questions)):
        for res in search(q, search_method):
            if i == res['id']:
                success += 1
                
    return success / len(questions)    

In [216]:
for method in ['word2vec', 'doc2vec', 'inverted_index']:
    print(method, eval_search_method(method))

1384it [09:31,  2.42it/s]


word2vec 1.778179190751445


1384it [17:31,  1.32it/s]


doc2vec 1.9234104046242775


1384it [15:30,  1.49it/s]


inverted_index 0.9913294797687862
